In [1]:
import os
import numpy as np
import pandas as pd
from utils.helper_functions import calculate_cbtmin, calculate_sleep_onset, calculate_sleep_duration

In [2]:
save_path = f"data/chang14_realistic_conditions"

In [3]:
selected_parameters = pd.read_csv(f"data/sleep_parameter_exploration/selected_sleep_parameters.csv",
                                  index_col=0)

In [4]:
time = np.load("data/light_schedules/time.npy")

In [5]:
DAILY_CONDITION_EBOOK_FIRST = [
    'None',
    'eBook', 'eBook', 'eBook', 'eBook', 'eBook',
    'None',
    'Book', 'Book', 'Book', 'Book', 'Book',
    'None',
]
DAILY_CONDITION_EBOOK_SECOND = [
    'None',
    'Book', 'Book', 'Book', 'Book', 'Book',
    'None',
    'eBook', 'eBook', 'eBook', 'eBook', 'eBook',
    'None',
]

DAILY_CONDITION_CHOICE = {
    'chang14_ebook_first': DAILY_CONDITION_EBOOK_FIRST,
    'chang14_ebook_second': DAILY_CONDITION_EBOOK_SECOND,
}

In [8]:
realistic_conditions = [
    (90, 1), (90, 2), (90, 3),
    (140, 4), (190, 4), (240, 4),
]
ebook_order_conditions = ['chang14_ebook_first', 'chang14_ebook_second']

In [12]:
simulation_features = pd.DataFrame()

for condition in realistic_conditions:
    indoor_light, reading_duration = condition
    simulation_data_path = f"data/chang14_realistic_conditions/{indoor_light}_lux_{reading_duration}_hours"
    for idx in selected_parameters.index:
        data = np.load(f"{simulation_data_path}/simulation_{indoor_light}_lux_{reading_duration}_hours_{idx}.npz")
        for reading_condition in ebook_order_conditions:
            sleep_onset = calculate_sleep_onset(time, data, reading_condition)
            cbtmin = calculate_cbtmin(time, data, reading_condition)
            sleep_duration = calculate_sleep_duration(time, data, reading_condition)

            min_length = min(len(sleep_onset), len(cbtmin))
            sleep_onset = sleep_onset[:min_length]
            cbtmin = cbtmin[:min_length]
            sleep_duration = sleep_duration[:min_length]

            normalized_sleep_onset = sleep_onset - np.arange(1, len(sleep_onset) + 1) * 24 + 24
            normalized_cbtmin = cbtmin - np.arange(1, len(cbtmin) + 1) * 24 + 24
            normalized_sleep_latency = (normalized_sleep_onset - 22.0) * 60
            normalized_sleep_latency = (sleep_onset % 24 - 22.0) * 60
            normalized_sleep_latency[normalized_sleep_latency < 0] = 0.0

            daily_condition = DAILY_CONDITION_CHOICE[reading_condition]

            data_subset = {
                'simulation_idx': idx,
                'day': np.arange(1, len(cbtmin) + 1),
                'simulation_condition': reading_condition,
                'sleep_onset': normalized_sleep_onset,
                'sleep_duration': sleep_duration,
                'cbtmin': normalized_cbtmin,
                'sleep_latency_minutes': normalized_sleep_latency,
                'daily_condition': daily_condition[:min_length],
                'indoor_light': indoor_light,
                'reading_duration': reading_duration,
            }
            simulation_features = pd.concat([simulation_features, pd.DataFrame(data_subset)])

In [14]:
simulation_features.to_csv(f"{save_path}/realistic_conditions_simulation_features.csv",
                           index=False)